In [1]:
#!pip install twython

In [2]:
import re
import string
import csv
import nltk
import sqlite3
import unicodedata

from sqlite3 import Error
from nltk.util import ngrams 
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *
from nltk.metrics import ConfusionMatrix
from sklearn.model_selection import train_test_split


lst_excessoes = None
lst_plural = None
lst_replace = None
lst_stopw = None
lst_base_treino = None
lst_add_base_treino = None
featureVectorBase = []
featureList=[]

conn = None

In [3]:
def processSentence(sentence):
    # process the tweets
    #Convert to lower case
    sentence = sentence.lower()
    #Convert www.* or https?://* to URL
    sentence = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',sentence)
    #Convert @username to AT_USER
    sentence = re.sub('@[^\s]+','AT_USER',sentence)
    #Remove additional white spaces
    sentence = re.sub('[\s]+', ' ', sentence)
    #Replace #word with word
    sentence = re.sub(r'#([^\s]+)', r'\1', sentence)
    #trimd
    sentence = sentence.strip('\'"')
    #retira açentos e caracteres especiais
    sentence = str(unicodedata.normalize('NFKD', sentence).encode('ascii','ignore'),'utf8')
    #************************* 
    for rpl in lst_replace:
        sentence = sentence.replace(rpl[0], rpl[1])

     
    return sentence

In [4]:
#start replaceTwoOrMore
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    #return pattern.sub(r"\1\1", s)
    return pattern.sub(r"\1", s)
#end

#start getStopWordList
def getStopWordList(stopWordListFileName):
    #read the stopwords file and build a list
    global lst_stopw
    stopWords = []
    
    if 1==1:
        stopWords = nltk.corpus.stopwords.words('portuguese')
    else:
        fp = open(stopWordListFileName, 'r')
        line = fp.readline()
        while line:
            word = line.strip()
            stopWords.append(word)
            line = fp.readline()
        fp.close()
        
    #****************************************
    for sw in lst_stopw:
        if sw[0]==None:
            stopWords.remove(sw[1])
            #print('remove: ' + str(sw[1]))
        else:
            stopWords.append(sw[0])
            #print('append: ' + str(sw[0]))
            
    return stopWords
#end

In [5]:
#start getfeatureVector
def getFeatureVector(sentence,stopWords):
    featureVector = []
    #split tweet into words
    words = sentence.split()
    for w in words:
        #replace two or more with two occurrences
        #w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.!')
        if len(w)>1:
            if 1!=1:
                #print(len(w))
                aux = w
                stemmer = nltk.stem.SnowballStemmer('portuguese')
                w = stemmer.stem(w)
                #if w=='indo':
                    #print(aux)
            #check if the word stats with an alphabet
            val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
            #ignore if it is a stop word
            if(w in stopWords or val is None):
                continue
            else:
                #replace two or more with two occurrences
                w = replaceTwoOrMore(w)
                #processa plural
                w = processa_plural(w)
                featureVector.append(w.lower())

    return featureVector
#end


In [6]:
def processa_plural(word):
    global lst_plural
    
    excessao_plural = [word] in [list(row) for row in lst_plural]
    
    if excessao_plural==False:
        if re.search(r's$', word):

            if re.search(r'as$', word):
                word = re.sub(r'irmas$', 'irma', word)
                word = re.sub(r'ais$', 'al', word)
                word = re.sub(r'aos$', 'ao', word)
                word = re.sub(r'as$', 'a', word)
                
            elif re.search(r'es$', word):
                if re.search(r'maes$', word):
                    word = 'maes'
                elif re.search(r'bres$', word):
                    word = re.sub(r'bres$', 'bre', word)
                elif re.search(r'zes$', word):
                    word = re.sub(r'zes$', 'z', word)
                elif re.search(r'tres$', word):
                    word = re.sub(r'tres$', 'tre', word)
                elif re.search(r'ores$', word):
                    word = re.sub(r'tres$', 'tre', word)
                else:
                    word = re.sub(r'oes$', 'ao', word)
                    word = re.sub(r'aes$', 'ao', word)
                    word = re.sub(r'gues$','gues', word)
                    word = re.sub(r'res$', 'r', word)
                    word = re.sub(r'es$', 'e', word)

            elif re.search(r'is$', word):
                if re.search(r'veis$', word):
                    word = re.sub(r'veis$', 'vel', word)
                else:
                    word = re.sub(r'ais$', 'al', word)
                    word = re.sub(r'zis$', 'zil', word)
                    word = re.sub(r'eis$', 'il', word)

            elif re.search(r'os$', word):
                if re.search(r'emos$', word):
                    word = re.sub(r'emos$', 'emos', word)
                elif re.search(r'amos$', word):
                    word = re.sub(r'amos$', 'amos', word)
                elif re.search(r'armos$', word):
                    word = re.sub(r'armos$', 'armos', word)
                else:
                    word = re.sub(r'ois$', 'oi', word)
                    word = re.sub(r'os$', 'o', word)

            elif re.search(r'us$', word):
                word = re.sub(r'eus$', 'eu', word)
                word = re.sub(r'us$', 'u', word)

            else:
                word = re.sub(r'ns$', 'm', word)

    return word
    

In [7]:
def extract_features(sentence):
    global featureList
    
    sentence_words = set(sentence)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in sentence_words)
    
    return features
#end

In [8]:
def features_old(sentence):
    global lst_excessoes
    
    words = sentence.lower().split()
    d={}

    if len(words)>1:
        for w in words:
            if len(w)>1:
                d['contains(%s)' % w]=True
    elif len(words)==1:
        excessao = words in [list(row) for row in lst_excessoes]
        if excessao==False:
            d['contains(%s)' % words[0]]=True

    return d

In [9]:
def carrega_listas_from_db():
    global lst_excessoes
    global lst_plural
    global lst_replace
    global lst_stopw
    global lst_base_treino
    global conn
    
    conn = sqlite3.connect('tweets.sqlite')
    cur = conn.cursor()
    #************************************************************************************
    cur.execute("SELECT de FROM excessoes where tipo = 'dic'")
    lst_excessoes = cur.fetchall()
    #************************************************************************************
    cur.execute("SELECT de FROM excessoes where tipo = 'pl'")
    lst_plural = cur.fetchall()
    #************************************************************************************
    cur.execute("SELECT de,para FROM excessoes where tipo = 'rpl'")
    lst_replace = cur.fetchall()
    #************************************************************************************
    cur.execute("SELECT de,para FROM excessoes where tipo = 'sw'")
    lst_stopw = cur.fetchall()
    #************************************************************************************
    cur.execute("SELECT * FROM tweet where tweet_id in (0,1)")
    lst_base_treino = cur.fetchall()
    
    conn.close()
    

In [10]:
def add_base_treino():
    global conn
    global lst_add_base_treino
   
    conn = sqlite3.connect('tweets.sqlite')
    cur = conn.cursor()
    cur.execute("SELECT reacao,sentimento FROM reacao where mark_for_train=1")
    lst_add_base_treino = cur.fetchall()
    
    if lst_add_base_treino!=None:
        conn = sqlite3.connect('tweets.sqlite')
        cur = conn.cursor()
        for row in lst_add_base_treino:
            cur.execute("insert into tweet (tweet_id, text, sentiment) values ('1', '" + str(row[0]) + "', '" + str(row[1]) + "')").fetchone()
            cur.execute("update reacao set mark_for_train=2 where mark_for_train=1")
            conn.commit()
    
    conn.close()

In [11]:
def word_grams(words,nro=2): 
    s = [] 
    maximo=nro+1
    
    #if len(words)==2:
    #    maximo=3
    #else:
    #    maximo=len(words)
    
    for n in range(1, maximo): 
        for ngram in ngrams(words, n): 
            s.append('_'.join(str(i) for i in ngram)) 
    
    return s 

In [12]:
def treina_modelo(ngrams=2):
    global lst_base_treino
    global featureVectorBase
    global featureList

    stopWords = getStopWordList('portuguese')
    featureList = []
    featureVector=[]
    featureVectorBase=[]

    # Get tweet words
    cont_neg=0
    cont_pos=0
    cont_neu=0
    cont_max=5000
    item_vec_max = 1

    for row in lst_base_treino:
        if len(row)==4:
            if row[3] == 1 and cont_pos <= cont_max:
                cont_pos +=1
                sentiment = row[3]
                sentence = row[2]                
                processedSentence = processSentence(sentence)
                featureVector = getFeatureVector(processedSentence, stopWords)
                if len(featureVector)>item_vec_max:
                    featureList.extend(featureVector)
                    featureVector = word_grams(featureVector,ngrams)
                    featureVectorBase.append((featureVector, sentiment))
                else:
                    #print(featureVector, sentiment)
                    1==1

            elif row[3] == -1 and cont_neg <= cont_max:
                cont_neg +=1
                sentiment = row[3]
                sentence = row[2]
                processedSentence = processSentence(sentence)
                featureVector = getFeatureVector(processedSentence, stopWords)
                if len(featureVector)>item_vec_max:
                    featureList.extend(featureVector)
                    featureVector = word_grams(featureVector,ngrams)
                    featureVectorBase.append((featureVector, sentiment))
                    1==1
                else:
                    #print(featureVector, sentiment)
                    1==1

            elif row[3] == 0 and cont_neu <= cont_max:
                cont_neu +=1
                sentiment = 1
                sentence = row[2]
                processedSentence = processSentence(sentence)
                featureVector = getFeatureVector(processedSentence, stopWords)
                if len(featureVector)>item_vec_max:
                    featureList.extend(featureVector)
                    featureVector = word_grams(featureVector,ngrams)
                    #featureVectorBase.append((featureVector, sentiment))
                    1==1
                else:
                    #print(featureVector, sentiment)
                    1==1
   

In [13]:
add_base_treino()

In [14]:
carrega_listas_from_db()

In [15]:
treina_modelo(ngrams=1)

In [16]:
featureList = list(set(featureList))

In [17]:
training_set = nltk.classify.util.apply_features(extract_features, featureVectorBase)

NAIVE BAYES CLASSIFIER

In [18]:
from nltk.classify import NaiveBayesClassifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [37]:
minDicLen = 0
test = 'bom dia! ainda não está no cardápio, mas fazemos sim, basta trocar o pão por omelete é só pedir aos garçons!'
processedTestTweet = processSentence(test)
print(processedTestTweet)
test=getFeatureVector(processedTestTweet,getStopWordList('portuguese'))
print(test)
test=word_grams(test,1)
print(test)
dic = extract_features(test)
#print(dic)
if len(dic)>minDicLen:
    print(classifier.classify(dic))
else:
    print(None)
    
classifier.prob_classify(dic).prob(1)

  !  ainda nao esta no cardapio, mas fazemos sim, basta trocar o pao por omelete e so pedir aos garcons ! 
['ainda', 'nao', 'cardapio', 'fazemos', 'sim', 'basta', 'trocar', 'pao', 'omelete', 'so', 'pedir', 'garcom']
['ainda', 'nao', 'cardapio', 'fazemos', 'sim', 'basta', 'trocar', 'pao', 'omelete', 'so', 'pedir', 'garcom']
-1


0.08415864189551837

In [20]:
print(classifier.show_most_informative_features(10000))

Most Informative Features
      contains(ridiculo) = True               -1 : 1      =     23.7 : 1.0
         contains(fraco) = True               -1 : 1      =     22.5 : 1.0
      contains(iritante) = True               -1 : 1      =     20.9 : 1.0
         contains(fraca) = True               -1 : 1      =     17.5 : 1.0
     contains(repeticao) = True               -1 : 1      =     15.3 : 1.0
         contains(erada) = True               -1 : 1      =     15.3 : 1.0
       contains(comecou) = True               -1 : 1      =     15.3 : 1.0
           contains(sol) = True               -1 : 1      =     14.2 : 1.0
      contains(monotono) = True               -1 : 1      =     12.5 : 1.0
           contains(sal) = True               -1 : 1      =     12.5 : 1.0
         contains(lenda) = True               -1 : 1      =     12.5 : 1.0
         contains(total) = True               -1 : 1      =     12.5 : 1.0
        contains(parado) = True               -1 : 1      =     12.5 : 1.0

       contains(ofender) = False               1 : -1     =      1.0 : 1.0
   contains(reacionario) = False               1 : -1     =      1.0 : 1.0
          contains(mini) = False               1 : -1     =      1.0 : 1.0
         contains(acuca) = False               1 : -1     =      1.0 : 1.0
          contains(bura) = False               1 : -1     =      1.0 : 1.0
      contains(idiotice) = False               1 : -1     =      1.0 : 1.0
    contains(incentivar) = False               1 : -1     =      1.0 : 1.0
     contains(produzido) = False               1 : -1     =      1.0 : 1.0
      contains(responde) = False               1 : -1     =      1.0 : 1.0
        contains(plagio) = False               1 : -1     =      1.0 : 1.0
    contains(propaganda) = False               1 : -1     =      1.0 : 1.0
contains(apaixonadisima) = False               1 : -1     =      1.0 : 1.0
        contains(gastei) = False               1 : -1     =      1.0 : 1.0
 contains(inconsistente) 

     contains(fanatismo) = False               1 : -1     =      1.0 : 1.0
     contains(lideranca) = False               1 : -1     =      1.0 : 1.0
          contains(sete) = False               1 : -1     =      1.0 : 1.0
      contains(brincava) = False               1 : -1     =      1.0 : 1.0
         contains(saude) = False               1 : -1     =      1.0 : 1.0
          contains(item) = False               1 : -1     =      1.0 : 1.0
       contains(perante) = False               1 : -1     =      1.0 : 1.0
      contains(desepero) = False               1 : -1     =      1.0 : 1.0
       contains(apavora) = False               1 : -1     =      1.0 : 1.0
          contains(five) = False               1 : -1     =      1.0 : 1.0
       contains(reclama) = False               1 : -1     =      1.0 : 1.0
      contains(resumise) = False               1 : -1     =      1.0 : 1.0
  contains(caracterizam) = False               1 : -1     =      1.0 : 1.0
          contains(vila) 

          contains(come) = False              -1 : 1      =      1.0 : 1.0
        contains(chance) = False              -1 : 1      =      1.0 : 1.0
         contains(sofre) = False              -1 : 1      =      1.0 : 1.0
     contains(descansei) = False              -1 : 1      =      1.0 : 1.0
     contains(destruida) = False              -1 : 1      =      1.0 : 1.0
        contains(mudado) = False              -1 : 1      =      1.0 : 1.0
       contains(servico) = False              -1 : 1      =      1.0 : 1.0
      contains(mantendo) = False              -1 : 1      =      1.0 : 1.0
     contains(terminado) = False              -1 : 1      =      1.0 : 1.0
     contains(fortisima) = False              -1 : 1      =      1.0 : 1.0
         contains(gemea) = False              -1 : 1      =      1.0 : 1.0
       contains(pasamos) = False              -1 : 1      =      1.0 : 1.0
        contains(mazela) = False              -1 : 1      =      1.0 : 1.0
     contains(primoroso) 

In [ ]:
print(classifier.most_informative_features(100000))

In [ ]:
test_result = []
gold_result = []

for i in range(0,len(FVnP_test)):
    #test_result.append(classifier.classify(FVP_test[i])
    test_result.append(str(classifier.classify(FVnP_test[i])))
    gold_result.append('False')
    if test_result != gold_result:
        dict = {}
        dict = FVnP_test[i]
        list_key = list(dict.keys())
        #print("Classificação: " + 'True')
        #print("Palavras".center(70,"*"))
        for c in range(0,len(list_key)):
            if dict.get(list_key[c])==True:
                #print(list_key[c]) 
                1==1
                
        #print("Fim".center(70,"*"))

CM = nltk.ConfusionMatrix(gold_result, test_result)
print(CM)    

In [ ]:
labels = {'True', 'False'}

from collections import Counter
TP, FN, FP = Counter(), Counter(), Counter()
for i in labels:
    for j in labels:
        if i == j:
            TP[i] += int(CM[i,j])
        else:
            FN[i] += int(CM[i,j])
            FP[j] += int(CM[i,j])

print("label\tprecision\trecall\tf_measure")
for label in sorted(labels):
    precision, recall = 0, 0
    if TP[label] == 0:
        f_measure = 0
    else:
        precision = float(TP[label]) / (TP[label]+FP[label])
        recall = float(TP[label]) / (TP[label]+FN[label])
        f_measure = float(2) * (precision * recall) / (precision + recall)
    print(label+"\t"+str(precision)+"\t"+str(recall)+"\t"+str(f_measure))
    